In [1]:
import pandas as pd
import numpy as np
import plotnine as p9
import pyfpgrowth as fp

import matplotlib.pyplot as plt
%matplotlib inline

import datetime
import time

In [2]:
# Reading JSON data into python as pandas dataframe.

data = pd.read_json('/Users/abdulrehman/Desktop/usml/data/AMAZON_FASHION.json',lines=True)
meta_data = pd.read_json('/Users/abdulrehman/Desktop/usml/data/meta_AMAZON_FASHION.json',lines=True)

# print(data)
# print(meta_data)

In [3]:
# Converting unix time to Date format and storing it in reviewTime Column.

data['reviewTime']=data.apply(lambda row: list(time.gmtime(row.unixReviewTime))[:-6], axis=1)
data = data.sort_values('unixReviewTime')
data['year']=data.apply(lambda row: row.reviewTime[0], axis=1)
# print(data)

In [ ]:
# Now we remove all customers who have made less than 5 transactions.
# It is hard to learn behaviour of a customer with just one or two transactions hence removing the customers with less
# transactions will help us learn better.

byreviewer = data.groupby('reviewerID')
byreviewer = byreviewer.filter(lambda x: len(x) > 4)
# print(type(byreviewer))
# print(byreviewer)

In [ ]:
# Now we take this cleaned data and create a Useritem matrix with users in columns and items as columns
# and fill it with each customer rating to the corresponding item column.

reviewers = list(byreviewer.reviewerID.unique())
# print(reviewers)
itemratedf = pd.DataFrame(data={'reviewerID':reviewers})
for index, row in byreviewer.iterrows():
#     print(index)
#     print(row['overall'])
#     print(row['reviewerID'])
#     print(row['asin'])
    itemratedf.loc[itemratedf['reviewerID'] == row['reviewerID'], row['asin']] = row['overall']

In [ ]:
# Saving the cleaned data frame and Useritem Matrix

itemratedf.to_csv(r'/Users/abdulrehman/Desktop/usml/data/AMAZON_FASHION_user_itemset.csv', index = False)
byreviewer.to_csv(r'/Users/abdulrehman/Desktop/usml/data/AMAZON_FASHION_clean.csv', index = False)

In [4]:
# Now we will prepare data to make recommendations only based on reviews which have 4 or more ratings.
# We will delete all the rows with overall ratings less than 4.
# Then we remove all customers who have made less than 5 transactions.
# It is hard to learn behaviour of a customer with just one or two transactions hence removing the customers with less
# transactions will help us learn better.

data.drop(data[ data['overall'] < 4 ].index , inplace=True)

byreviewer = data.groupby('reviewerID')
byreviewer = byreviewer.filter(lambda x: len(x) > 4)

In [5]:
# Now we take this cleaned data and create a Useritem matrix with users in columns and items as columns
# and fill it with each customer rating to the corresponding item column.

reviewers = list(byreviewer.reviewerID.unique())
itemratedf = pd.DataFrame(data={'reviewerID':reviewers})
for index, row in byreviewer.iterrows():
    itemratedf.loc[itemratedf['reviewerID'] == row['reviewerID'], row['asin']] = row['overall']

In [6]:
# Saving the cleaned data frame and Useritem Matrix of Positive Reviews only dataset.

itemratedf.to_csv(r'/Users/abdulrehman/Desktop/usml/data/AMAZON_FASHION_posonly_user_itemset.csv', index = False)
byreviewer.to_csv(r'/Users/abdulrehman/Desktop/usml/data/AMAZON_FASHION_posonly_user_clean.csv', index = False)

In [ ]:
# We will now do EDA on the cleaned Data in EDA.ipynb